In [2]:
import sys, os
from pyspark.sql.functions import  monotonically_increasing_id
from delta.tables import DeltaTable

# Insert the parent directory (one level up) onto Python’s module search path

from  helper_methods_stocks  import DevSparkSession 
from helper_methods_stocks import SCDType2Handler


ImportError: cannot import name 'DevSparkSession' from 'helper_methods_stocks' (/Users/PC/Desktop/VS Code Repositories/azure-stock-market/src/helper_methods_stocks/__init__.py)

### Initiate Spark

In [ ]:

# stockdata = GetStockData() 
spark = DevSparkSession().spark


silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table
dim_symbol_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/dim-symbol"
fact_daily_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Gold/delta-tables/fact-daily-summary"



## Dim Symbol Loader

In [ ]:


parameters = {
        "businessColumns" : "Symbol,ExchangeName,Currency",
        "typeIColumns" : "", 
        "tableType" : "Dim"
        }
scd2Handler =  SCDType2Handler(parameters)



df_silver = spark.read.format("delta").load(silver_path)
df_dimSymbol =  df_silver.select("Symbol","ExchangeName","Currency", "Type", "ExchangeTimeZone").distinct()
scd2Handler.refresh_timestamp()

# df_dimSymbol = df_dimSymbol.withColumn("Type", concat(col("Type"), lit("_test") ) )

add_audit_columns =  scd2Handler.add_audit_columns
df_dimSymbol = df_dimSymbol.transform(add_audit_columns)
sid_offest = spark.read.format("delta").load(dim_symbol_path)
sid_offest = sid_offest.selectExpr("max(SymbolSID)").head()[0]
sid_offest = sid_offest + 1 if sid_offest else 0



df_dimSymbol = df_dimSymbol.withColumn("SymbolSID", monotonically_increasing_id() +  sid_offest)
spark.read.format("delta").load(dim_symbol_path).show()
df_dimSymbol.show(truncate=False)
deltaTable = DeltaTable.forPath(spark, dim_symbol_path)
scd2Handler.delta_merge_typeII(deltaTable, df_dimSymbol)


In [ ]:
# df_history = spark.sql(f" select *  from  delta.`{dim_symbol_path}`  ")

  
# df_history.show(truncate=False)